# Implementations of the `exp` function

In [1]:
num_eval = 1000
x_eval = 2.

## C++

Let's use this Jupyter notebook to write a `cpp` file with an implementation of the `exp` function

In [2]:
%%file cexp_func.cpp
#include <cstdio>
#include <cstdlib>
#include <cmath>

extern "C" // required when using C++ compiler
double sum_exp(int nmax, double x){
    double sum = 1.0;
    double element = 1.0;
    // summation
    for(int n=1; n<=nmax; n++) {
        element *= x / static_cast<double>(n);
        sum += element;
    }
    return sum;
}


int main(int argc, char* argv[]){
    int nmax = 1;
    if(argc > 1)
        nmax = atoi(argv[1]);
    for(int i=2; i<argc; i++){
        double x = atof(argv[i]);
        double tmp = sum_exp(nmax, x);
        printf("exp( %.2f ) = %.4f (error: %.e)\n", x, tmp, exp(x)-tmp);
    }
    return 0;
}


Overwriting cexp_func.cpp


Now, we can compile the generate file and execute it:

In [3]:
!g++ -O3 -o cexp_func cexp_func.cpp
!./cexp_func 10 1 2 -3

exp( 1.00 ) = 2.7183 (error: 3e-08)
exp( 2.00 ) = 7.3890 (error: 6e-05)
exp( -3.00 ) = 0.0533 (error: -4e-03)


Note that the first argument is `nmax` followed by the arguments at which to evaluate the `exp` function.

In [4]:
!g++ -O3 -fPIC -shared -o cexp_func.so cexp_func.cpp

In [5]:
from ctypes import *
clib = CDLL("./cexp_func.so")
sp = clib.sum_exp
sp.argtypes = [c_int, c_double]
sp.restype = c_double
%timeit sp(num_eval, x_eval)

1.7 µs ± 34.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Python implementation

### Plain Python
Let's write the same function in Python.

In [6]:
def sum_exp(nmax, x):
    sum = 1.0;
    element = 1.0;
    for n in range(1,nmax+1):
        element *= x / n
        sum += element;
    return sum;

In [7]:
import numpy as np
def unittest_sum_exp(nmax, x):
    diff = sum_exp(nmax, x) - np.exp(x) 
    if np.isclose(diff, 0.):
        print("unit test passed")
    else:
        raise ValueError("something's wrong with the Python implementation of `exp`")
unittest_sum_exp(num_eval, x_eval)

unit test passed


In [8]:
%timeit sum_exp(num_eval, x_eval)

56.4 µs ± 1.89 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### with Numba

You might need to install numba first:
```shell
!python3 -m pip install numba
```

In [9]:
import numba as nb
@nb.jit(nopython=True, fastmath=True)
def sum_exp(nmax, x):
    sum = 1.0;
    element = 1.0;
    for n in range(1,nmax+1):
        element *= x / n
        sum += element;
    return sum;
sum_exp(num_eval, x_eval);

In [10]:
%timeit sum_exp(num_eval, x_eval)

4.57 µs ± 162 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Fortran

In [11]:
%%file fexp_func.f90
subroutine fexp_func (nmax, x, sum)
  integer, intent(in) :: nmax
  real(8), intent(in)  :: x
  real(8), intent(out) :: sum 
  real(8) :: element 
  sum = 1.d0
  element = 1.d0      
  do n = 1, nmax, 1    
     element = element * x / float(n)
     sum = sum + element
  end do
end subroutine fexp_func

Overwriting fexp_func.f90


In [12]:
!f2py --opt='-O3' -c -m fexp_func fexp_func.f90

running build
running config_cc
INFO: unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
INFO: unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
INFO: build_src
INFO: building extension "fexp_func" sources
INFO: f2py options: []
INFO: f2py:> /var/folders/kn/6_p1475d0wd6sbc0jmc9t8jr0000gn/T/tmpdu36w36g/src.macosx-13-x86_64-3.11/fexp_funcmodule.c
creating /var/folders/kn/6_p1475d0wd6sbc0jmc9t8jr0000gn/T/tmpdu36w36g/src.macosx-13-x86_64-3.11
Reading fortran codes...
	Reading file 'fexp_func.f90' (format:free)
Post-processing...
	Block: fexp_func
			Block: fexp_func
Applying post-processing hooks...
  character_backward_compatibility_hook
Post-processing (stage 2)...
Building modules...
    Building module "fexp_func"...
    Generating possibly empty wrappers"
    Maybe empty "fexp_func-f2pywrappers.f"
        Constructing wrapper function "fexp_func"...
          sum = fexp_func(nmax,x)

In [13]:
import fexp_func
%timeit fexp_func.fexp_func(num_eval, x_eval)

4.7 µs ± 171 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
